<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Removing Duplicates**


Estimated time needed: **30** minutes


## Introduction


In this lab, you will focus on data wrangling, an important step in preparing data for analysis. Data wrangling involves cleaning and organizing data to make it suitable for analysis. One key task in this process is removing duplicate entries, which are repeated entries that can distort analysis and lead to inaccurate conclusions.  


## Objectives


In this lab you will perform the following:


1. Identify duplicate rows  in the dataset.
2. Use suitable techniques to remove duplicate rows and verify the removal.
3. Summarize how to handle missing values appropriately.
4. Use ConvertedCompYearly to normalize compensation data.
   


### Install the Required Libraries


In [1]:
!pip install pandas

### Step 1: Import Required Libraries


In [2]:
import pandas as pd

### Step 2: Load the Dataset into a DataFrame



load the dataset using pd.read_csv()


In [3]:
# Define the URL of the dataset
file_path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"

# Load the dataset into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows to ensure it loaded correctly
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

**Note: If you are working on a local Jupyter environment, you can use the URL directly in the <code>pandas.read_csv()</code>  function as shown below:**



#df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv")


### Step 3: Identifying Duplicate Rows


**Task 1: Identify Duplicate Rows**
  1. Count the number of duplicate rows in the dataset.
  2. Display the first few duplicate rows to understand their structure.


In [4]:
# Step 3: Identify duplicate rows
total_duplicates = df.duplicated().sum()
print("Total exact duplicate rows:", total_duplicates)

if total_duplicates > 0:
    dup_rows = df[df.duplicated(keep='first')]
    print("\nShowing first 10 duplicate rows:")
    display(dup_rows.head(10))
else:
    print("No exact duplicate rows found.")


Total exact duplicate rows: 0
No exact duplicate rows found.


### Step 4: Removing Duplicate Rows


**Task 2: Remove Duplicates**
   1. Remove duplicate rows from the dataset using the drop_duplicates() function.
2. Verify the removal by counting the number of duplicate rows after removal .


In [5]:
# Step 4: Remove duplicate rows
df_no_dup = df.drop_duplicates(keep='first').reset_index(drop=True)
removed_count = df.shape[0] - df_no_dup.shape[0]
print(f"Removed {removed_count} duplicate rows. New shape: {df_no_dup.shape}")

# verify
print("Duplicates remaining (should be 0):", df_no_dup.duplicated().sum())

# replace working df with deduped version (optional)
df = df_no_dup


Removed 0 duplicate rows. New shape: (65437, 114)
Duplicates remaining (should be 0): 0


### Step 5: Handling Missing Values


**Task 3: Identify and Handle Missing Values**
   1. Identify missing values for all columns in the dataset.
   2. Choose a column with significant missing values (e.g., EdLevel) and impute with the most frequent value.


In [6]:
# Step 5: Handling missing values
print("Missing counts (descending):")
missing_counts = df.isnull().sum().sort_values(ascending=False)
display(missing_counts[missing_counts > 0])

# choose column to impute: prefer 'EdLevel'
col_to_impute = 'EdLevel' if 'EdLevel' in df.columns else (missing_counts.idxmax() if missing_counts.max() > 0 else None)

if col_to_impute is None:
    print("No column with missing values to impute.")
else:
    print("Column chosen for imputation:", col_to_impute)
    mode_vals = df[col_to_impute].mode(dropna=True)
    if mode_vals.empty:
        print("No mode available for this column (all values may be NaN). Skipping imputation.")
    else:
        mode_val = mode_vals.iloc[0]
        before = df[col_to_impute].isnull().sum()
        df[col_to_impute].fillna(mode_val, inplace=True)
        after = df[col_to_impute].isnull().sum()
        print(f"Imputed {col_to_impute} missing values with mode = {mode_val!r}. Before: {before}, After: {after}")


Missing counts (descending):


AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
YearsCode                      5568
NEWSOSites                     5151
LearnCode                      4949
EdLevel                        4653
AISelect                       4530
Length: 109, dtype: int64

Column chosen for imputation: EdLevel
Imputed EdLevel missing values with mode = 'Bachelor’s degree (B.A., B.S., B.Eng., etc.)'. Before: 4653, After: 0


/tmp/ipykernel_300/2128675424.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col_to_impute].fillna(mode_val, inplace=True)


### Step 6: Normalizing Compensation Data


**Task 4: Normalize Compensation Data Using ConvertedCompYearly**
   1. Use the ConvertedCompYearly column for compensation analysis as the normalized annual compensation is already provided.
   2. Check for missing values in ConvertedCompYearly and handle them if necessary.


In [7]:
# Step 6: Normalize/check ConvertedCompYearly
if 'ConvertedCompYearly' in df.columns:
    df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')
    missing_after_coerce = df['ConvertedCompYearly'].isnull().sum()
    print("ConvertedCompYearly missing after coercion:", missing_after_coerce)
    median_val = df['ConvertedCompYearly'].median(skipna=True)
    print("Median ConvertedCompYearly:", median_val)
    if pd.notna(median_val):
        df['ConvertedCompYearly'].fillna(median_val, inplace=True)
        print("Filled missing ConvertedCompYearly with median.")
    print("\nConvertedCompYearly summary:")
    display(df['ConvertedCompYearly'].describe())
else:
    print("Column 'ConvertedCompYearly' not found — skipping normalization step.")


ConvertedCompYearly missing after coercion: 42002
Median ConvertedCompYearly: 65000.0
Filled missing ConvertedCompYearly with median.

ConvertedCompYearly summary:


/tmp/ipykernel_300/1171113345.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConvertedCompYearly'].fillna(median_val, inplace=True)


count    6.543700e+04
mean     7.257636e+04
std      1.122207e+05
min      1.000000e+00
25%      6.500000e+04
50%      6.500000e+04
75%      6.500000e+04
max      1.625660e+07
Name: ConvertedCompYearly, dtype: float64

### Step 7: Summary and Next Steps


**In this lab, you focused on identifying and removing duplicate rows.**

- You handled missing values by imputing the most frequent value in a chosen column.

- You used ConvertedCompYearly for compensation normalization and handled missing values.

- For further analysis, consider exploring other columns or visualizing the cleaned dataset.


In [ ]:
# Step 7: Summary & verification
print("Final dataframe shape:", df.shape)
print("Exact duplicates remaining:", df.duplicated().sum())

print("\nColumns with remaining missing values (if any):")
remaining_missing = df.isnull().sum().sort_values(ascending=False)
display(remaining_missing[remaining_missing > 0])

print("\nQuick checks you can run next:")
print(" - df.head()")
print(" - df.info()")
print(" - df.describe(include='all')")


<!--
## Change Log

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2024-11-05|1.2|Madhusudhan Moole|Updated lab|
|2024-09-24|1.1|Madhusudhan Moole|Updated lab|
|2024-09-23|1.0|Raghul Ramesh|Created lab|

--!>


Copyright © IBM Corporation. All rights reserved.
